## Quickly work out the fitness corresponding to proposal objective functions for example models

In [237]:
import sys
import os
import numpy as np
import pickle
import pandas as pd
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
import itertools
import qinfer as qi
from lfig import LatexFigure
import copy
import random 

sys.path.append("/home/bf16951/QMD")
import qmla

%matplotlib inline


In [268]:
class obj_model():
    def __init__(
        self, 
        model_instance, 
        idx = 'a', 
        num_samples = 100,
    ):
        
        self.model_instance = model_instance
        self.model_id = self.model_instance.model_id
        self.f_score = q.model_database[
            q.model_database.model_id == self.model_instance.model_id
        ].f_score.values[0]
        self.likelihoods = model_instance.evaluation_normalization_record
        self.norm_rec = [np.log(l) for l in self.likelihoods]
        self.log_likelihood = sum(self.norm_rec)
        self.log_likelihood = model_instance.evaluation_log_likelihood       
        self.likelihood_avg = np.median(self.likelihoods)
        self.likelihood_std_dev = np.std(self.likelihoods)
        self.num_bf_wins = 0
        self.num_samples = num_samples
        self.residuals = self.model_instance.evaluation_pr0_diffs
        
        self.sum_residuals = np.sum(self.residuals)
        self.mean_residual = (np.mean(self.residuals))
        self.num_terms = self.model_instance.num_terms
        
        self.name = r"$\hat{{H}}_{}$".format(idx)
        self.idx = idx
        
        
        self.aic = 2*self.num_terms - 2*self.log_likelihood
        self.aicc = (
            self.aic 
            + (
                2*self.num_terms*(self.num_terms+1)
                /(self.num_samples-self.num_terms-1)
            )
        )
    
        self.inverse_ll = -1/self.log_likelihood
        
        self.bic = (
            self.num_terms * np.log(self.num_samples)
            - 2 * self.log_likelihood
        )
        
        self.bayes_wt = (
            np.e**(-0.5*self.bic)
        )
        
    def latex_command(
        self, 
        obj='aic'
    ):
        string = "\\newcommand{{\\obj{obj}{m}}}{{{f}}}".format(
            obj = obj, 
            m = self.idx, 
            f = np.round(self.__getattribute__(obj), 2)
        )
        
        return string
    

In [295]:
def get_akaike_weights(set_models):
    aicc_values = {
        i : set_models[i].aicc
        for i in set_models.keys()
    }
    min_aicc = min(aicc_values.values())
    
    akaike_wts =  {
        set_models[i].name : np.e**((min_aicc - aicc_values[i])/2 )
        for i in set_models.keys()
    }
    return akaike_wts

def get_percentages(
    objective_fncs, 
    target_metric,
    num_to_truncate_to=4,
#     to_round=True, 
):
    subset = objective_fncs.reset_index()
    method = subset[
        subset.Metric == target_metric
    ].Method.unique()[0] # should only correspond to a single Method

    subset = subset[subset.Metric == target_metric]

    subset.drop(columns=['Metric', 'Method'], inplace=True)
    if 'index' in subset.keys():
        subset.drop(columns=['index'], inplace=True)

    transposed = subset.T

    acceptance_cutoff = sorted(transposed.values, reverse=True)[:num_to_truncate_to][-1]
    transposed[transposed.values < acceptance_cutoff] = 0 

    all_pts = subset.values.sum()
    probs = np.array(transposed.values/all_pts)
    probs = np.round(probs.astype(np.double), 2)
    probs *= 100
    transposed['prob'] = probs

    transposed = transposed.T

    transposed['Metric'] = r"$\%$"
    transposed['Method'] = method
    
    return pd.Series(transposed.iloc[1])

In [453]:
results = qmla.load_results(
    results_folder = "/home/bf16951/bc_results/",
    results_time = "Dec_08/20_41",
    instance_id = 7 # use 3 for thesis/paper as it gives clear arguments
)
q = results['qmla_instance']
es = results['exploration_strategy']

In [454]:
sorted_model_ids = iter(q.model_database[
    ["f_score", "model_id"]
].sort_values("f_score").model_id.values)

In [455]:
example_models={
    'a' : obj_model( # F-score 0
        model_instance = q.get_model_storage_instance_by_id(next(sorted_model_ids)), 
        idx='a', 
    ),
    'b' : obj_model( # F-score 0.2
        model_instance = q.get_model_storage_instance_by_id(next(sorted_model_ids)),
        idx='b',
    ),
    'c' : obj_model( # F-score 0.4
        model_instance = q.get_model_storage_instance_by_id(next(sorted_model_ids)),
        idx='c', 
    ),
    'd' : obj_model( # F-score 0.5
        model_instance = q.get_model_storage_instance_by_id(next(sorted_model_ids)),
        idx='d'
    ),
    'e' : obj_model( # F-score 0.7
        model_instance = q.get_model_storage_instance_by_id(next(sorted_model_ids)),
        idx='e'
    ),
    'f' : obj_model( # F-score 0.8
        model_instance = q.get_model_storage_instance_by_id(next(sorted_model_ids)),
        idx='f'
    ),
}


In [456]:
available_models = sorted(example_models.keys())
model_pairs = list(itertools.combinations(available_models, r=2))
subset_model_pairs = random.sample(
    model_pairs, 
    len(model_pairs) # include all comparisons since there are so few
    #     int(len(model_pairs)/2)+2 
)
model_comparisons = qmla.utilities.flatten(subset_model_pairs)
model_comparison_counts = {idx : model_comparisons.count(idx) for idx in example_models}

print("Model comparison counts:", model_comparison_counts)

Model comparison counts: {'d': 5, 'c': 5, 'e': 5, 'b': 5, 'a': 5, 'f': 5}


In [457]:
rating_system = qmla.shared_functionality.rating_system.ModifiedEloRating()
bayes_factors = {}

for model in example_models.values():
    model.num_bf_wins = 0

for i, j in subset_model_pairs:
    mod_i = example_models[i]
    mod_j = example_models[j]
#     ll_i = mod_i.log_likelihood
#     ll_j = mod_j.log_likelihood
    
#     bf = np.e**(ll_i - ll_j)

    bf = q.bayes_factors_df[
        (q.bayes_factors_df.id_a == mod_i.model_id)
        & (q.bayes_factors_df.id_b == mod_j.model_id)
    ].bayes_factor.values[0]
    
    if bf > 1: 
        mod_i.num_bf_wins += 1
    elif bf < 1:
        mod_j.num_bf_wins += 1
    
    bayes_factors[(i,j)] = bf

# update using BF    
rating_system.batch_update(
    bayes_factors
)
for i in available_models:
    r = rating_system.models[i].rating
    example_models[i].elo_rating = int(r)

    
# just get them from the update
for i in example_models:
    mid = example_models[i].model_instance.model_id
    r = int(es.ratings_class.models[mid].rating)
    example_models[i].elo_rating = int(r)

Ratings batch update for spawn step  0
rating df models: {}
Models: ['e', 'c', 'd', 'b', 'a', 'f'] 
 already present: []
Ratings of present models []
Adding e with starting rating 1000
Adding c with starting rating 1000
Adding d with starting rating 1000
Adding b with starting rating 1000
Adding a with starting rating 1000
Adding f with starting rating 1000
Rating update. A/B=c/e 	 BF=1.463202526568762e-09
Rating update. A/B=b/f 	 BF=1.6532759291535552e-86
Rating update. A/B=d/f 	 BF=1.134795173618263e-22
Rating update. A/B=a/e 	 BF=1.868058283981836e-29
Rating update. A/B=a/f 	 BF=1.3133778788746698e-49
Rating update. A/B=b/e 	 BF=1.303635907701836e-19
Rating update. A/B=a/b 	 BF=1.7271118847576112e+38
Rating update. A/B=c/d 	 BF=546733155126723.0
Rating update. A/B=a/c 	 BF=2.0974187151271813e-30
Rating update. A/B=e/f 	 BF=1.9321018508046823e-15
Rating update. A/B=a/d 	 BF=2.5210600688889312e-15
Rating update. A/B=b/d 	 BF=1.3925564511736257e-51
Rating update. A/B=d/e 	 BF=2.6678878

In [458]:
# Points by ranking
model_points = {
    mod.idx : mod.num_bf_wins
    for mod in example_models.values()
}

ranked_model_list = sorted(
    model_points,
    key=model_points.get,
    reverse=True
)

num_to_truncate_to = 4
num_zero_pt_models = len(ranked_model_list) - num_to_truncate_to
rankings= [0]*num_zero_pt_models
rankings.extend(
    list(range(1, 1+num_to_truncate_to))
)

# rankings = list(range(1, len(ranked_model_list) + 1))
rankings.reverse()

num_points = sum(rankings) # number of points to distribute
ranking_points = list(zip(
    ranked_model_list, 
    [r/num_points for r in rankings]
))
ranking_points = dict(ranking_points)

In [459]:
for m in ranking_points:
    example_models[m].ranking_points = ranking_points[m]
    example_models[m].ranking = ranked_model_list.index(m)+1

In [460]:
cols = ['Method', 'Metric']
cols.extend([example_models[m].name for m in available_models])

objective_fncs = pd.DataFrame(
    columns = cols
)

# F score
g = {
    example_models[m].name : 
        r"${}$".format(
            np.round(example_models[m].f_score, 1)
        )
    for m in available_models
}
g['Method'] = ''
g['Metric'] = '$F_1$'
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)

# number terms 
g = {
    example_models[m].name : example_models[m].num_terms
    for m in available_models
}
g['Method'] = ''
g['Metric'] = '$k$'
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)

# average likelihood
g = {
    example_models[m].name : 
        r"${} \pm {}$".format(
            np.round(example_models[m].likelihood_avg, 2),
            np.round(example_models[m].likelihood_std_dev,2)            
        )
    for m in available_models
}
g['Method'] = ''
g['Metric'] = '$\overline{l_e}$'
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)


# log-likelihood
g = {
    example_models[m].name : example_models[m].log_likelihood
    for m in available_models
}
g['Method'] = ''
g['Metric'] = '$\tll_i$'
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)

# 1/LL
g = {
    example_models[m].name : example_models[m].inverse_ll
    for m in available_models
}
g['Method'] = 'Inverse log-likelihood'
g['Metric'] = '$g_i^L$'
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)

probs = get_percentages(
    objective_fncs, 
    target_metric = g['Metric']
)
objective_fncs.loc[len(objective_fncs)] = probs

# AIC
g = {
    example_models[m].name : example_models[m].aic
    for m in available_models
}
g['Method'] = 'Akaike Info Criterion'
g['Metric'] = 'AIC'
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)

# AICC
g = {
    example_models[m].name : example_models[m].aicc
    for m in available_models
}
g['Method'] = 'Akaike Info Criterion'
g['Metric'] = 'AICc'
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)

# Akaike weights
g = get_akaike_weights(example_models)
g['Method'] = 'Akaike Info Criterion'
g['Metric'] = r"$w_i^A$"
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)


g = {
    example_models[m].name : (1/example_models[m].aicc)**2
    for m in available_models
}
g['Method'] = 'Akaike Info Criterion'
g['Metric'] = r"$g_i^A$"
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)


probs = get_percentages(
    objective_fncs, 
    target_metric = g['Metric']
)
objective_fncs.loc[len(objective_fncs)] = probs

# Bayesian Info Criterion
g = {
    example_models[m].name : example_models[m].bic
    for m in available_models
}
g['Method'] = 'Bayesian Info Criterion'
g['Metric'] = 'BIC'
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)

g = {
    example_models[m].name : example_models[m].bayes_wt
    for m in available_models
}
g['Method'] = 'Bayesian Info Criterion'
g['Metric'] = r"w_i^B" 
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)

g = {
    example_models[m].name : (1/example_models[m].bic)**2
    for m in available_models
}
g['Method'] = 'Bayesian Info Criterion'
g['Metric'] = r"g_i^B" 
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)

probs = get_percentages(
    objective_fncs, 
    target_metric = g['Metric']
)
objective_fncs.loc[len(objective_fncs)] = probs

# Number wins
g = {
    example_models[m].name : example_models[m].num_bf_wins
    for m in available_models
}
g['Method'] = 'Bayes factor points'
g['Metric'] = 'g_i^p'
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)
objective_fncs.loc[len(objective_fncs)] = get_percentages(
    objective_fncs, 
    target_metric = g['Metric']
)

# Ranking points
g = {
    example_models[m].name : example_models[m].ranking
    for m in available_models
}
g['Method'] = 'Ranking points'
g['Metric'] = 'Ranking'
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)


g = {
    example_models[m].name : example_models[m].ranking_points
    for m in available_models
}
g['Method'] = 'Ranking points'
g['Metric'] = 'g_i^R'
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)

objective_fncs.loc[len(objective_fncs)] = get_percentages(
    objective_fncs, 
    target_metric = g['Metric']
)

# Elo ratings
g = {
    example_models[m].name : "{}".format(np.round(example_models[m].elo_rating) )
    for m in available_models
}
min_elo_rating = min([float(r) for r in g.values()])
g['Method'] = 'Elo rating'
g['Metric'] = 'Rating'
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)

g = {
    example_models[m].name : example_models[m].elo_rating - min_elo_rating
    for m in available_models
}
g['Method'] = 'Elo rating'
g['Metric'] = 'g_i^E'
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)

objective_fncs.loc[len(objective_fncs)] = get_percentages(
    objective_fncs, 
    target_metric = g['Metric']
)


# Residuals
g = {
    example_models[m].name : example_models[m].mean_residual
    for m in available_models
}
g['Method'] = 'Residuals'
g['Metric'] = r'mean$\{\tilde{r_p^e}\}$'
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)

g = {
    example_models[m].name : (1 - np.mean(example_models[m].residuals))**2
    for m in available_models
}
g['Method'] = 'Residuals'
g['Metric'] = 'g_i^r'
objective_fncs.loc[len(objective_fncs)] = pd.Series(g)

objective_fncs.loc[len(objective_fncs)] = get_percentages(
    objective_fncs, 
    target_metric = g['Metric']
)

# Set multi-colulmn index
objective_fncs.set_index(['Method', 'Metric'], inplace=True)

In [461]:
col_fmt = 'c|'*len(example_models)
col_fmt = "|cc|" + col_fmt
# pd.set_option('display.float_format', '{:.2g}'.format)

table = objective_fncs.to_latex(
    buf=None, 
    bold_rows=False,
    float_format="{:7.3g}".format,
#     float_format="{:0.3E}".format,
    escape=False,
    multirow=True,
    column_format=col_fmt,
    index_names=True,
    col_space = 40, 
)

table = table.replace("\\toprule", "\\hline")
table = table.replace("\\bottomrule", "\\hline")
table = table.replace("\\midrule", "")
table = table.replace("\\multirow", "\\hline \\multirow")
table = table.replace("Metric", "")

# for save_dir in [
#     "/home/bf16951/thesis/theoretical_study/figures",
#     "/home/bf16951/theory_paper/figure_development"
# ]:
#     buf=os.path.join(
#         save_dir, 
#         'obj_fnc_table.tex',
#     )

#     with open(buf, 'w') as f:
#         f.write(table)

In [462]:
objective_fncs

$\hat{H}_a$  \
Method                  Metric                                     
                        $F_1$                              $0.0$   
                        $k$                                    3   
                        $\overline{l_e}$         $0.82 \pm 0.31$   
                        $\tll_i$                          -147.9   
Inverse log-likelihood  $g_i^L$                       0.00676133   
                        $\%$                                   0   
Akaike Info Criterion   AIC                                301.8   
                        AICc                              302.05   
                        $w_i^A$                       7.2582e-11   
                        $g_i^A$                      1.09608e-05   
                        $\%$                                   0   
Bayesian Info Criterion BIC                              309.616   
                        w_i^B                        5.85931e-68   
                        g_i^B                        1.04317e-05   
                        $\%$                                  23   
Bayes factor points     g_i^p                                  1   
                        $\%$                                   0   
Ranking points          Ranking                                5   
                        g_i^R                                  0   
                        $\%$                                   0   
Elo rating              Rating                               965   
                        g_i^E                                 75   
                        $\%$                                   0   
Residuals               mean$\{\tilde{r_p^e}\}$         0.130862   
                        g_i^r                             0.7554   
                        $\%$                                   0   

                                                     $\hat{H}_b$  \
Method                  Metric                                     
                        $F_1$                              $0.2$   
                        $k$                                    4   
                        $\overline{l_e}$         $0.82 \pm 0.33$   
                        $\tll_i$                          -187.9   
Inverse log-likelihood  $g_i^L$                       0.00532198   
                        $\%$                                   0   
Akaike Info Criterion   AIC                                383.8   
                        AICc                             384.221   
                        $w_i^A$                      1.04139e-28   
                        $g_i^A$                      6.77388e-06   
                        $\%$                                   0   
Bayesian Info Criterion BIC                              394.221   
                        w_i^B                        2.48924e-86   
                        g_i^B                        6.43459e-06   
                        $\%$                                   0   
Bayes factor points     g_i^p                                  0   
                        $\%$                                   0   
Ranking points          Ranking                                6   
                        g_i^R                                  0   
                        $\%$                                   0   
Elo rating              Rating                               890   
                        g_i^E                                  0   
                        $\%$                                   0   
Residuals               mean$\{\tilde{r_p^e}\}$          0.12776   
                        g_i^r                           0.760803   
                        $\%$                                  23   

                                                     $\hat{H}_c$  \
Method                  Metric                                     
                        $F_1$                              $0.4$   
                        $k$            

In [309]:
# Get model names for direct use in text -> wrap in \begin{align}
print(
    "\hat{{H}}_0 \ &= \ {};\\\\".format(q.exploration_class.true_model_latex())
)
for k in sorted(example_models): 
    print(
        "\hat{{H}}_{} \ &= \ {}; \\\\".format(k, example_models[k].model_instance.model_name_latex)
    )

\hat{H}_0 \ &= \ $\sigma_{(1, 2)}^{z}\sigma_{(1, 3)}^{z}\sigma_{(2, 3)}^{z}\sigma_{(2, 5)}^{z}\sigma_{(3, 5)}^{z}$;\\
\hat{H}_a \ &= \ $\sigma_{(1, 3)}^{z}\sigma_{(1, 4)}^{z}\sigma_{(1, 5)}^{z}\sigma_{(2, 4)}^{z}\sigma_{(2, 5)}^{z}\sigma_{(3, 4)}^{z}\sigma_{(3, 5)}^{z}$; \\
\hat{H}_b \ &= \ $\sigma_{(1, 4)}^{z}\sigma_{(1, 5)}^{z}\sigma_{(2, 5)}^{z}\sigma_{(3, 4)}^{z}$; \\
\hat{H}_c \ &= \ $\sigma_{(1, 5)}^{z}\sigma_{(3, 4)}^{z}\sigma_{(4, 5)}^{z}$; \\
\hat{H}_d \ &= \ $\sigma_{(1, 2)}^{z}\sigma_{(1, 5)}^{z}\sigma_{(2, 4)}^{z}\sigma_{(2, 5)}^{z}\sigma_{(4, 5)}^{z}$; \\
\hat{H}_e \ &= \ $\sigma_{(1, 2)}^{z}\sigma_{(1, 3)}^{z}\sigma_{(2, 3)}^{z}\sigma_{(2, 4)}^{z}\sigma_{(2, 5)}^{z}\sigma_{(3, 4)}^{z}\sigma_{(3, 5)}^{z}$; \\
\hat{H}_f \ &= \ $\sigma_{(1, 2)}^{z}\sigma_{(1, 3)}^{z}\sigma_{(1, 5)}^{z}\sigma_{(2, 3)}^{z}\sigma_{(2, 5)}^{z}\sigma_{(4, 5)}^{z}$; \\


# Table showing Elo comparison between two models

In [ ]:
class rated_model():
    def __init__(
        self, 
        init_rating = 1000,
        idx = 'a'
    ):
        self.rating = init_rating
        self.name =  r"$\hat{{H}}_{}$".format(idx)
        
        
    def expected_outcome(
        self, 
        opponent_rating
    ):
        b = (opponent_rating - self.rating)/400
        ex = 1 / (1 + 10**b)
        return ex
    
    
    def update(
        self, 
        opponent_rating, 
        bf,
        update_idx, 
    ):
        
        if bf > 1: 
            s = 1
        else: 
            s = 0 
            
            
        expected_outcome = self.expected_outcome(opponent_rating)
        
        delta_rating = s - expected_outcome
        
        weight = np.abs(np.log10(bf))
        old_rating = self.rating
        self.rating = self.rating + (weight * delta_rating)
        print("BF = {} => S={}. D={}; k={} => R={}".format(bf, s, delta_rating, weight, self.rating))
        
        
        self.summary = pd.Series({
            'Update' : update_idx, 
            'Model' : self.name,
            r'$R_i$' : old_rating, 
            r'$B_{ij}$' : bf, 
            r"$log_{10}(B_{ij})$" : weight, 
            r"$S_i$" : s, 
            r"$E_i$": expected_outcome, 
            r"$\Delta R_i$" : delta_rating, 
            r"$R_i^{\prime}$" : "{}".format(np.round(self.rating))
            
        })
        return self.summary
        
        
        

In [ ]:
elo_table = pd.DataFrame(columns = [
    'Update', 
    'Model',
    r'$R_i$',
    r'$E_i$',
    r'$S_i$',
    r'$B_{ij}$',
    r'$log_{10}(B_{ij})$',
    r'$\Delta R_i$',
    r'$R_i^{\prime}$',
])


bf = 1e100 # in favour of stronger model

bf_list = [bf, 1/bf]
descriptions = [r"$\hat{H}_a > \hat{H}_b$", r"$\hat{H}_b > \hat{H}_a$"]
for bf in bf_list:
    idx = bf_list.index(bf)
    update_idx = descriptions[idx]

    rmod1 = rated_model(init_rating = 1000, idx='a')
    rmod2 = rated_model(init_rating = 800, idx = 'b')

    r1_init_rating = rmod1.rating
    r2_init_rating = rmod2.rating

    rmod1.update(opponent_rating = r2_init_rating, bf = bf, update_idx = update_idx)
    rmod2.update(opponent_rating = r1_init_rating, bf = 1/bf, update_idx = update_idx)
    
    elo_table.loc[len(elo_table)] = rmod1.summary
    elo_table.loc[len(elo_table)] = rmod2.summary

In [ ]:
num_cols = len(elo_table.keys()) - 2

col_fmt = 'c|'*num_cols
col_fmt = "|cc|" + col_fmt


# elo_table.drop(columns=[]'index')
elo_table.set_index(['Update', 'Model'], inplace=True)

In [ ]:
buf=os.path.join(
    os.getcwd(), 
    'elo_table.tex',
)


# col_fmt = 'c|'*len(example_models)
# col_fmt = "|cc|" + col_fmt

table = elo_table.to_latex(
    buf=None, 
    bold_rows=False,
    float_format="{:0.3g}".format,
    escape=False,
    multirow=True,
    column_format=col_fmt,
    index_names=True,
    col_space = 40, 
)

table = table.replace("\\toprule", "\\hline")
table = table.replace("\\bottomrule", "\\hline")
table = table.replace("\\multirow", "\\hline \\multirow")
table = table.replace("Update", "")

with open(buf, 'w') as f:
    f.write(table)

In [ ]:
np.log(10*100)